# Generate the Dataset

We want to predict possible future evolutions of the pandemic knowing the parameters of the simulator which more or less have been able to describe the trend of the real data for the past year.
These parameters have been computed in the previous notebook.

In [1]:
import json
import pandas as pd
import covasim as cv
import seaborn as sns
from pyDOE import lhs
from src.data import get_regional_data
from src.interventions import get_sampling_interventions

with open('../res/parameters.json', 'r') as json_file:
  j = json.load(json_file)

initial_params = j['initial_params']
intervention_params = j['intervention_params']
df = get_regional_data(4.46e6 / initial_params['pop_size'])

Covasim 2.0.3 (2021-03-11) — © 2021 by IDM


## Latin-Hypercube Sampling

In [2]:
n_zones_interv = 8
samples = lhs(n=n_zones_interv, samples=2)
samples = pd.DataFrame(samples)
for i in range(n_zones_interv):
    samples[i] = samples[i].map(lambda v: int(4 * v)).map({0: 'W', 1: 'Y', 2: 'O', 3: 'R'})
samples

,0,1,2,3,4,5,6,7
0,R,O,R,O,R,R,Y,Y
1,Y,Y,W,Y,Y,W,O,O


We will consider a shorter time interval, in particular, around a month centered in today.

In [3]:
time_interval = 20 # days after which put a zone
result_cols = ['hosp', 'diag', 'dead', 'susc']
sim_days = range(0, time_interval * 2)
data_cols = [col + '_' + str(day) for day in sim_days for col in result_cols]

In [ ]:
# 251 = (2020-11-01) - (2020-02-24)
zones_starting_day = 251
initial_params['n_days'] = zones_starting_day + samples.shape[1] * time_interval

data = []
for idx, s in samples.iterrows():
    print("Generating samples for simulation", idx)
    intervs = get_sampling_interventions(s, intervention_params , time_interval)
    sim = cv.Sim(pars={**initial_params, 'rand_seed': idx}, interventions=intervs, datafile=df)
    msim = cv.MultiSim(sim)
    msim.run(n_runs=1)
    msim.mean()
    result = pd.DataFrame()
    result['hosp'] = msim.results['n_severe'].values + msim.results['n_critical'].values
    result['diag'] = msim.results['new_diagnoses'].values
    # result['diag'] = result['diag'].rolling(7).mean()
    result['dead'] = msim.results['new_deaths'].values
    # result['dead'] = result['dead'].rolling(7).mean()
    result['susc'] = msim.results['n_susceptible'].values
    result = result.iloc[zones_starting_day+1:].reset_index(drop=True)
    for i in range(0, len(result) - time_interval, time_interval):
        slc = result.iloc[i : i + 2*time_interval]
        raw = [slc[key].values for key in result_cols]
        raw = [item for daily in zip(*raw) for item in daily]
        row = pd.Series(raw)
        data.append(row)

Generating samples for simulation 0
Generating samples for simulation 1


Finally, we can collect the results in a dataframe.

In [10]:
pd.options.display.max_columns = 160

df = pd.concat(data, axis=1).transpose()
df.columns = data_cols
df

,hosp_0,diag_0,dead_0,susc_0,hosp_1,diag_1,dead_1,susc_1,hosp_2,diag_2,dead_2,susc_2,hosp_3,diag_3,dead_3,susc_3,hosp_4,diag_4,dead_4,susc_4,hosp_5,diag_5,dead_5,susc_5,hosp_6,diag_6,dead_6,susc_6,hosp_7,diag_7,dead_7,susc_7,hosp_8,diag_8,dead_8,susc_8,hosp_9,diag_9,dead_9,susc_9,hosp_10,diag_10,dead_10,susc_10,hosp_11,diag_11,dead_11,susc_11,hosp_12,diag_12,dead_12,susc_12,hosp_13,diag_13,dead_13,susc_13,hosp_14,diag_14,dead_14,susc_14,hosp_15,diag_15,dead_15,susc_15,hosp_16,diag_16,dead_16,susc_16,hosp_17,diag_17,dead_17,susc_17,hosp_18,diag_18,dead_18,susc_18,hosp_19,diag_19,dead_19,susc_19,hosp_20,diag_20,dead_20,susc_20,hosp_21,diag_21,dead_21,susc_21,hosp_22,diag_22,dead_22,susc_22,hosp_23,diag_23,dead_23,susc_23,hosp_24,diag_24,dead_24,susc_24,hosp_25,diag_25,dead_25,susc_25,hosp_26,diag_26,dead_26,susc_26,hosp_27,diag_27,dead_27,susc_27,hosp_28,diag_28,dead_28,susc_28,hosp_29,diag_29,dead_29,susc_29,hosp_30,diag_30,dead_30,susc_30,hosp_31,diag_31,dead_31,susc_31,hosp_32,diag_32,dead_32,susc_32,hosp_33,diag_33,dead_33,susc_33,hosp_34,diag_34,dead_34,susc_34,hosp_35,diag_35,dead_35,susc_35,hosp_36,diag_36,dead_36,susc_36,hosp_37,diag_37,dead_37,susc_37,hosp_38,diag_38,dead_38,susc_38,hosp_39,diag_39,dead_39,susc_39
0,98.0,71.714286,1.285714,373463.0,106.0,79.000000,1.285714,373393.0,107.0,77.857143,1.571429,373313.0,116.0,77.285714,1.285714,373234.0,125.0,85.571429,1.000000,373187.0,134.0,84.142857,1.142857,373140.0,137.0,82.000000,1.285714,373094.0,149.0,80.714286,1.285714,373054.0,157.0,73.285714,1.428571,373015.0,165.0,67.714286,1.428571,372976.0,165.0,61.000000,1.714286,372953.0,161.0,49.571429,1.857143,372915.0,152.0,46.571429,2.000000,372892.0,142.0,45.857143,2.142857,372871.0,138.0,42.714286,2.428571,372840.0,127.0,38.285714,2.571429,372816.0,113.0,35.285714,3.000000,372799.0,106.0,32.857143,2.857143,372787.0,101.0,31.428571,2.857143,372771.0,93.0,26.857143,2.428571,372748.0,94.0,24.571429,2.000000,372713.0,87.0,24.000000,1.714286,372687.0,69.0,21.714286,1.571429,372659.0,62.0,21.000000,0.857143,372622.0,59.0,19.000000,1.000000,372577.0,57.0,17.571429,0.857143,372527.0,57.0,16.142857,0.857143,372490.0,57.0,15.142857,0.857143,372439.0,54.0,14.142857,0.714286,372391.0,50.0,14.571429,0.428571,372331.0,47.0,16.142857,0.285714,372290.0,43.0,17.142857,0.142857,372222.0,38.0,18.000000,0.428571,372167.0,35.0,21.571429,0.571429,372112.0,31.0,22.000000,0.857143,372036.0,31.0,24.000000,0.857143,371969.0,38.0,25.285714,1.000000,371916.0,38.0,24.857143,1.000000,371811.0,40.0,29.285714,0.857143,371741.0,43.0,31.714286,0.714286,371656.0
1,94.0,24.571429,2.000000,372713.0,87.0,24.000000,1.714286,372687.0,69.0,21.714286,1.571429,372659.0,62.0,21.000000,0.857143,372622.0,59.0,19.000000,1.000000,372577.0,57.0,17.571429,0.857143,372527.0,57.0,16.142857,0.857143,372490.0,57.0,15.142857,0.857143,372439.0,54.0,14.142857,0.714286,372391.0,50.0,14.571429,0.428571,372331.0,47.0,16.142857,0.285714,372290.0,43.0,17.142857,0.142857,372222.0,38.0,18.000000,0.428571,372167.0,35.0,21.571429,0.571429,372112.0,31.0,22.000000,0.857143,372036.0,31.0,24.000000,0.857143,371969.0,38.0,25.285714,1.000000,371916.0,38.0,24.857143,1.000000,371811.0,40.0,29.285714,0.857143,371741.0,43.0,31.714286,0.714286,371656.0,44.0,31.000000,0.571429,371580.0,47.0,32.571429,0.285714,371495.0,45.0,33.857143,0.714286,371402.0,44.0,37.428571,0.714286,371313.0,43.0,42.428571,1.000000,371234.0,49.0,41.714286,1.000000,371159.0,54.0,43.000000,1.000000,371074.0,56.0,46.428571,1.142857,370977.0,62.0,48.000000,1.142857,370872.0,66.0,46.714286,0.714286,370768.0,73.0,46.857143,0.714286,370670.0,79.0,49.571429,0.428571,370563.0,78.0,52.142857,0.857143,370460.0,79.0,50.571429,0.857143,370380.0,80.0,46.857143,0.857143,370270.0,87.0,45.714286,1.000000,370146.0,86.0,50.142857,1.571429,370028.0,87.0,54.714286,1.571429,369916.0,95.0,55.428571,1.857143,369801.0,96.0,55.428571,1.714286,369693.0
2,44.0,31.000000,0.571429,371580.0,47.0,32.571429,0.285714,371495.0,45.0,33.857143,0.714286,37140